# 2D Tomographic Image Reconstruction with FBP and MLEM

## Learning objectives of this notebook

1. Understanding / exploring the discrete Radon transform (forward projection) and its adjoint / transpose (back projections).
2. Understanding / exploring 2D tomographic image reconstruction using filtered back projection (FBP)
3. Understanding / exploring iterative 2D tomographic image reconstruction using Maximum Likelihood Expectation Maximization (MLEM)
4. Exploring the convergence behavior of MLEM using noise-free and noisy data sinograms.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from utils import DiscreteRadonTransform, demo_sinogram, demo_image

# limit the number of elemets printed for very long arrays
np.set_printoptions(threshold=6, edgeitems=3)

# use interactive plots
%matplotlib widget

We create a linear operator to calculate the discrete Radon transform (line integrals through a discrete 2D image), commonly called "forward projection." 
Additionally, this operator can compute the transpose (adjoint) of the Radon transform, often referred to as "back projection."

In [ ]:
discrete_radon_transform = DiscreteRadonTransform()

The discrete 2D image resides on a 2D pixel grid. Below, we display the image shape and the x and y world coordinates of the pixel grid.

In [ ]:
print(f"image shape (num_x, num_y) : {discrete_radon_transform.image_shape}")
print(f"pixel x coordinates        : {discrete_radon_transform.pixel_x_coordinates}")
print(f"pixel y coordinates        : {discrete_radon_transform.pixel_y_coordinates}")

The discrete Radon transform (a set of line integrals) is represented as a sinogram with radial (s) and angular (theta) axes.
Below, we display the shape of the sinogram and the radial and angular coordinates of its pixels.

In [ ]:
print(f"sinogram shape (num_s, num_theta): {discrete_radon_transform.sinogram_shape}")
print(f"sinogram radial coordinates      : {discrete_radon_transform.s}")
print(f"sinogram angular coordinates     : {discrete_radon_transform.theta}")

### Forward Projection of a Centered Point

In the two cells below, we create an image that is zero everywhere except at the central pixel, where we assign a value of 1.
We then compute the discrete Radon transform of this object (the forward projection):

$$
p(s,\theta) = R[f(x,y)]
$$

In [ ]:
# Let's create an image full of zeros everywhere except for a single pixel in the center.
# this is called "f(x,y)" in the above formula
center_point_image = np.zeros(discrete_radon_transform.image_shape, dtype=np.float32)
center_point_image[discrete_radon_transform.image_shape[0] // 2, discrete_radon_transform.image_shape[1] // 2] = 1.0

In [ ]:
# Let's calculate the discrete Radon transform of the image
# this is called "p(s,\theta)" in the above formula
sinogram_center_point = discrete_radon_transform(center_point_image)

Let's visualize the image of the centered point and its corresponding Radon transform (the sinogram).
In the image shown on the left, all values are zero except for a small point at the center, located at world coordinates (x=0, y=0), corresponding to pixel (i=100, j=100).

In [ ]:
# Let's show the center point image and its radon transform (sinogram)

image_plot_kws = dict(cmap="Grays", origin="lower", extent=discrete_radon_transform.image_extent)
sinogram_plot_kws = dict(
    cmap="Grays", origin="lower", extent=discrete_radon_transform.sinogram_extent, aspect=20
)

fig, ax = plt.subplots(1, 2, figsize=(8, 4), tight_layout=True)
im0 = ax[0].imshow(center_point_image.T, **image_plot_kws)
im1 = ax[1].imshow(sinogram_center_point.T, **sinogram_plot_kws)
ax[0].set_xlabel(discrete_radon_transform.image_axis_labels[0])
ax[0].set_ylabel(discrete_radon_transform.image_axis_labels[1])
ax[1].set_xlabel(discrete_radon_transform.sinogram_axis_labels[0])
ax[1].set_ylabel(discrete_radon_transform.sinogram_axis_labels[1])
ax[0].set_title(r"Center point image $f(x,y)$", fontsize="medium")
ax[1].set_title(r"Radon transform $p(s,\theta) = R[f(x,y)]$", fontsize="medium")
fig.colorbar(im0, ax=ax[0], location="bottom", fraction=0.03)
fig.colorbar(im1, ax=ax[1], location="bottom", fraction=0.03)

## Task 1.1

(1.1) Create an image of a point that is off-center. The image should be zero everywhere except at the off-center pixel (i=150, j=100), which corresponds to the world coordinates (x=15, y=0). 
Compute the discrete Radon transform of this image and visualize both the image and its Radon transform in a figure with two plots, as shown previously for the centered point source.


In [ ]:
# ADD YOUR CODE FOR TASK 1.1 HERE
#
# offcenter_point_image = ...
# sinogram_offcenter_point = ...
# 
# fig2, ax2 = plt.subplots(1, 2, figsize=(8, 4), tight_layout=True)
# ...

# YOUR CODE HERE
raise NotImplementedError()

## Task 1.2

(1.2) What do you observe in the plot of the Radon transform of the off-center point image? 
How does it compare to the Radon transform of the centered point image?

YOUR ANSWER HERE

## Understanding the Transpose of the Radon Transform (Back Projection)

In lecture 02, we learned that it is possible to calculate the transpose of the Radon transform, commonly referred to as backprojection. 
In the cells below, we:
1. create a test image, $f(x,y)$,
2. compute the Radon transform of the test image (the forward projection), $p(s,\theta) = R[f(x,y)]$, and
3. compute the transpose of the Radon transform (the backprojection) of the forward projection of the image, $g(x,y) = R^T[ R[f(x,y)]]$.

In [ ]:
# load a test image
test_image0 = demo_image(discrete_radon_transform.pixel_x_coordinates, discrete_radon_transform.pixel_y_coordinates, image_id=0)
# calculate the discrete Radon transform of the test image
discrete_radon_transform_test_image0 = discrete_radon_transform(test_image0)
# back project the Radon transform of the test image
back_projected_discrete_radon_transform_test_image0 = discrete_radon_transform.adjoint(discrete_radon_transform_test_image0)

Let's visualize the test image, its forward projection and the back projection of the forward projection.

In [ ]:
# this scale factor is needed due to discretization and makes sure that the
# back projections are shown with the correct scale
scale_factor = discrete_radon_transform.dtheta / (discrete_radon_transform.pixel_size[0] * discrete_radon_transform.pixel_size[1])

fig3, ax3 = plt.subplots(1, 3, figsize=(12, 4), tight_layout=True)
im03 = ax3[0].imshow(test_image0.T, **image_plot_kws)
im13 = ax3[1].imshow(discrete_radon_transform_test_image0.T, **sinogram_plot_kws)
im23 = ax3[2].imshow(scale_factor * back_projected_discrete_radon_transform_test_image0.T, **image_plot_kws)
ax3[0].set_xlabel(discrete_radon_transform.image_axis_labels[0])
ax3[0].set_ylabel(discrete_radon_transform.image_axis_labels[1])
ax3[1].set_xlabel(discrete_radon_transform.sinogram_axis_labels[0])
ax3[1].set_ylabel(discrete_radon_transform.sinogram_axis_labels[1])
ax3[2].set_xlabel(discrete_radon_transform.image_axis_labels[0])
ax3[2].set_ylabel(discrete_radon_transform.image_axis_labels[1])
ax3[0].set_title(r"image $f(x,y)$", fontsize="medium")
ax3[1].set_title(r"$p(s,\theta) = R[f(x,y)]$", fontsize="medium")
ax3[2].set_title(r"$g(x,y) = R^T[R[f(x,y)]]$", fontsize="medium")
fig3.colorbar(im03, ax=ax3[0], location="bottom", fraction=0.03)
fig3.colorbar(im13, ax=ax3[1], location="bottom", fraction=0.03)
fig3.colorbar(im23, ax=ax3[2], location="bottom", fraction=0.03)

## Task 2.1

(2.1) What do you observe in the backprojection of the forward projection? 
Is the backprojection the inverse of the Radon transform? 
In other words, can a simple backprojection be used for tomographic image reconstruction?


YOUR ANSWER HERE

## Task 2.2


(2.2) The back projection of the forward projection can be described as a 2D convolution of the input image with a specific kernel in 
image space? Which kernel is that?

YOUR ANSWER HERE

## Image Reconstruction using Filtered Back Projection (FBP)

As we have learned in lecture 02, one way to invert the Radon transform is to use filtered back projection (FBP). In a FBP we back project a sinogram that was "ramp filtered" in the radial direction.

$$
f(x,y) = R^H[q(s,\theta)]
$$

where $q(s, \theta)$ is the ramp filtered sinogram defined by 

$$
q(s,\theta) = p(s,\theta) \ast h(s) = \mathcal{F}_{1D}^{-1}[\mathcal{F}_{1D}[p(s,\theta)] \, \hat{h}(\nu)]
$$

with 

$$
\hat{h}(\nu) = \begin{cases}
|\nu|, \ \text{if} \ |\nu| \leq \nu_{max}  \\
0, \ \text{otherwise}
\end{cases}
$$

To test the FBP algorithm, we first a load noise-free test sinogram in the cell below.

In [ ]:
# load noise-free test sinogram
noise_free_sinogram1 = demo_sinogram(discrete_radon_transform.s, discrete_radon_transform.theta, image_id=1, gamma=0.0)

Before implementing and applying the filtered back projection, we also apply a normal back projection of the sinogram as above.

In [ ]:
# back project noise free sinogram
back_projected_noise_free_sinogram1 = discrete_radon_transform.adjoint(noise_free_sinogram1)

## Task 3.1

(3.1) Implement the ramp filter that we need for FBP by completing the function in the cell below. The input to this function is a single 1D projection profile $p(s)$ for a given $\theta$ (a "row" in a sinogram). The output is the ramp-filtered version of the profile $q(s)$.

**Hints:**
1. The ramp filter can be implemented either via a multiplication in Fourier space **or** by a convolution in projection space.
2. The derivation of the ramp filter in projection space is given [here](https://kul-recon-lab.github.io/course_nucmed_tech/appendix-one#app-ramp). Convolutions can be calculated using [numpy's convolve function](https://numpy.org/doc/stable/reference/generated/numpy.convolve.html).
3. For working in Fourier space, you can use [numpy's fast Fourier transform](https://numpy.org/doc/2.1/reference/routines.fft.html).

In [ ]:
# setup a discrete ramp filter in image space
def ramp_filter_projection_profile(projection_profile):
    """Discrete ramp filter of a single 1D projection profile

    Parameters
    ----------

    projection_profile ... real 1D numpy array
       containg a projection profile
       p(s) and fixed \theta in the above notation

    Returns
    -------

    real 1D numpy array
    containing the ramp filtered projection profile
    q(s) for fixed \theta in the above notation

    Note
    ----

    The ramp filter can be implemented in projection profile
    space using a convolution **or**
    in Fourier space using a multiplication.
    """

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
def ramp_filter_sinogram(sinogram):
    """ramp filter a complete 2D sinogram by applying the filter ramp filter for each projection profile"""
    ramp_filtered_sinogram = np.zeros_like(sinogram)

    # loop over the angular dimension of the sinogram
    for i in range(sinogram.shape[1]):
        # apply the ramp filter for a given projection profile
        ramp_filtered_sinogram[:, i] = ramp_filter_projection_profile(sinogram[:, i])

    return ramp_filtered_sinogram

Let's apply the ramp filter to the first projection profile of the noise free sinogram and visualize the input, filter and ouput

In [ ]:
test_profile = noise_free_sinogram1[:,0]
ramp_filtered_test_profile = ramp_filter_projection_profile(test_profile)

fig3b, ax3b = plt.subplots(1,2, figsize = (7,3.5), tight_layout = True, sharex = True)
ax3b[0].plot(discrete_radon_transform.s, test_profile)
ax3b[1].plot(discrete_radon_transform.s, ramp_filtered_test_profile)
ax3b[0].set_xlabel(r"$s$")
ax3b[1].set_xlabel(r"$s$")
ax3b[0].set_title(r"$p(s,0)$")
ax3b[1].set_title(r"$q(s,0) = p(s,0) \ast h(s)$")
ax3b[0].grid(ls = ":")
ax3b[1].grid(ls = ":")

Let's apply our implemented ramp filter to the complete noise free test sinogram

In [ ]:
ramp_filtered_noise_free_sinogram1 = ramp_filter_sinogram(noise_free_sinogram1)

filtered_back_projected_noise_free_sinogram1 = discrete_radon_transform.adjoint(
    ramp_filtered_noise_free_sinogram1
)

Let's visualize the: noise free test sinogram, the ramp filtered version of this sinogram, and the back projections of the latter two

In [ ]:
fig4, ax4 = plt.subplots(2, 2, figsize=(8, 8), tight_layout=True, sharex = "col", sharey = "col")
im04 = ax4[0, 0].imshow(noise_free_sinogram1.T, **sinogram_plot_kws)
im14 = ax4[0, 1].imshow(
    scale_factor * back_projected_noise_free_sinogram1.T, **image_plot_kws
)
im24 = ax4[1, 0].imshow(ramp_filtered_noise_free_sinogram1.T, **sinogram_plot_kws)
im34 = ax4[1, 1].imshow(
    scale_factor * filtered_back_projected_noise_free_sinogram1.T, **image_plot_kws
)
ax4[0, 0].set_xlabel(discrete_radon_transform.sinogram_axis_labels[0])
ax4[0, 0].set_ylabel(discrete_radon_transform.sinogram_axis_labels[1])
ax4[0, 1].set_xlabel(discrete_radon_transform.image_axis_labels[0])
ax4[0, 1].set_ylabel(discrete_radon_transform.image_axis_labels[1])
ax4[1, 0].set_xlabel(discrete_radon_transform.sinogram_axis_labels[0])
ax4[1, 0].set_ylabel(discrete_radon_transform.sinogram_axis_labels[1])
ax4[1, 1].set_xlabel(discrete_radon_transform.image_axis_labels[0])
ax4[1, 1].set_ylabel(discrete_radon_transform.image_axis_labels[1])
ax4[0, 0].set_title(r"acquired sinogram $y$", fontsize="medium")
ax4[0, 1].set_title(r"back projection of sinogram $R^T y$", fontsize="medium")
ax4[1, 0].set_title(r"ramp filtered acquired sinogram $y \ast h$", fontsize="medium")
ax4[1, 1].set_title(
    r"back projection of filtered sinogram $R^T(y \ast h)$", fontsize="medium"
)
fig4.colorbar(im04, ax=ax4[0, 0], location="bottom", fraction=0.03)
fig4.colorbar(im14, ax=ax4[0, 1], location="bottom", fraction=0.03)
fig4.colorbar(im24, ax=ax4[1, 0], location="bottom", fraction=0.03)
fig4.colorbar(im34, ax=ax4[1, 1], location="bottom", fraction=0.03)

## Iterative image reconstruction using MLEM

In lecture 03, we have learned that we can also iteratively invert the Radon transform (reconstruct the image) using the MLEM update given by

$$
x^{(n+1)} = \frac{x^{(n)}}{A^T 1} A^T \left( \frac{y}{A x^{(n)}} \right)
$$

where:
1. $x^{(n)}$ is the image at iteration $n$
2. $y$ is the (measured) sinogram to be reconstructed
3. $A$ is the linear physics forward model - in our case here the discrete Radon transform $R$
4. $A^T$ is the adjoint (transpose) of the linear physics forward model
5. $A^T 1$ is the adjoint of the linear physics forward model applied to a sinogram full of ones. This image is called the senstivity image

## Task 4.1

(4.1) Implement the MLEM update of the formula above, by completing the function in the cell below.
The input the the update function are:
1. the `current_image` ($x^{(n)}$)
2. the `data_sinogram` to be reconstructed ($y$)
3. the `linear_physics_operator` ($A$)
4. the `sensitivity_image` ($A^T 1$)

The output the function is supposed to be the MLEM updated image $(x^{(n+1)})$.

In [ ]:
def my_mlem_update(current_image, data_sinogram, linear_physics_operator, sensitivity_image):
    # YOUR CODE HERE
    raise NotImplementedError()

Let's run MLEM using 100 iterations and store all 100 iterates as well as the initialization.

In [ ]:
# number of MLEM iterations
num_iterations = 100

# precalculate the sensitivity image A^T 1
sens_image = discrete_radon_transform.adjoint(np.ones(discrete_radon_transform.sinogram_shape, dtype=np.float32))

# allocate an array for all 50 MLEm updates and the initialization 
mlem_iterates = np.zeros((num_iterations + 1,) + discrete_radon_transform.image_shape, dtype=np.float32)

# initialize with the 0-th MLEM iterate with a disk full of 0.5
X, Y = np.meshgrid(discrete_radon_transform.pixel_x_coordinates, discrete_radon_transform.pixel_y_coordinates, indexing="ij")
R = np.sqrt(X**2 + Y**2)
mlem_iterates[0, ...][R <= discrete_radon_transform.s.max()] = 0.5

# run the MLEM updates
for i in range(num_iterations):
    print(f"{(i+1):03}/{num_iterations:03}", end="\r")
    mlem_iterates[i + 1, ...] = my_mlem_update(
        current_image=mlem_iterates[i, ...],
        data_sinogram=noise_free_sinogram1,
        linear_physics_operator=discrete_radon_transform,
        sensitivity_image=sens_image,
    )

Let's visualize the MLEM updates and also the convergence of the two pixels:
1. (i=100, j=115) corresponding to the world coordinates (x=0, y=4.4)
2. (i=67, j=100) corresponding to the world coordinates (x=-10, y=0)

In [ ]:
# show the MLEM iterates
fig5, ax5 = plt.subplots(
    4, 4, figsize=(12, 12), tight_layout=True, sharex="row", sharey="row"
)

# show a few selected MLEM updates
for i, it in enumerate([0, 1, 2, 3, 4, 5, 6, 7, 10, 20, 40, num_iterations]):
    im = ax5.ravel()[i].imshow(mlem_iterates[it].T, **image_plot_kws, vmax=1.75)
    ax5.ravel()[i].set_title(f"MLEM iterate {it}", fontsize="medium")
    ax5.ravel()[i].set_xlabel(discrete_radon_transform.image_axis_labels[0])
    ax5.ravel()[i].set_ylabel(discrete_radon_transform.image_axis_labels[1])
    fig5.colorbar(im, ax=ax5.ravel()[i], location="bottom", fraction=0.03, pad = 0.2)

# show the converges of the in two selected pixels
ax5[-1, 0].plot(mlem_iterates[:, 100, 115])
ax5[-1, 1].plot(mlem_iterates[:, 67, 100])

for axx in ax5[-1, :2]:
    axx.set_xlabel("MLEM iteration")
    axx.set_ylabel("pixel value")
    axx.grid(ls=":")
ax5[-1, 0].set_title("convergence pixel (100, 115)", fontsize="medium")
ax5[-1, 1].set_title("convergence pixel (67, 100)", fontsize="medium")
for axx in ax5[-1, 2:]:
    axx.set_axis_off()

## Task 5.1

(5.1) What do you observe regarding the convergence of the values in the two pixels `(i=150,j=172)` and `(i=100,j=150)`? How do the values after 25 iterations compare to the values after 75 iterations? Can you explain potential differences in the convergence between the two pixels?

YOUR ANSWER HERE

## Reconstruction of noisy sinograms using FBP and MLEM

In the following, we will reconstruct sinograms suffering from Poisson noise using FBP and MLEM.
In the cell below we load a noisy version of the test sinogram.

In [ ]:
# load noisy test sinogram
noisy_sinogram1 = demo_sinogram(discrete_radon_transform.s, discrete_radon_transform.theta, image_id=1, gamma=1.0)

Let's calculate a back projection as well as a filtered back projection of the noisy sinogram

In [ ]:
back_projected_noisy_sinogram1 = discrete_radon_transform.adjoint(noisy_sinogram1)

ramp_filtered_noisy_sinogram1 = ramp_filter_sinogram(noisy_sinogram1)

filtered_back_projected_noisy_sinogram1 = discrete_radon_transform.adjoint(
    ramp_filtered_noisy_sinogram1
)

Let's visualize the noisy sinogram, it's ramp-filtered version and the back projection of the latter two.

In [ ]:
fig6, ax6 = plt.subplots(2, 2, figsize=(8, 8), tight_layout=True, sharex = "col", sharey = "col")
im06 = ax6[0, 0].imshow(noisy_sinogram1.T, **sinogram_plot_kws)
im16 = ax6[0, 1].imshow(
    scale_factor * back_projected_noisy_sinogram1.T, **image_plot_kws
)
im26 = ax6[1, 0].imshow(ramp_filtered_noisy_sinogram1.T, **sinogram_plot_kws)
im36 = ax6[1, 1].imshow(
    scale_factor * filtered_back_projected_noisy_sinogram1.T, **image_plot_kws
)
ax6[0, 0].set_xlabel(discrete_radon_transform.sinogram_axis_labels[0])
ax6[0, 0].set_ylabel(discrete_radon_transform.sinogram_axis_labels[1])
ax6[0, 1].set_xlabel(discrete_radon_transform.image_axis_labels[0])
ax6[0, 1].set_ylabel(discrete_radon_transform.image_axis_labels[1])
ax6[1, 0].set_xlabel(discrete_radon_transform.sinogram_axis_labels[0])
ax6[1, 0].set_ylabel(discrete_radon_transform.sinogram_axis_labels[1])
ax6[1, 1].set_xlabel(discrete_radon_transform.image_axis_labels[0])
ax6[1, 1].set_ylabel(discrete_radon_transform.image_axis_labels[1])
ax6[0, 0].set_title(r"acquired sinogram $y$", fontsize="medium")
ax6[0, 1].set_title(r"back projection of sinogram $R^T y$", fontsize="medium")
ax6[1, 0].set_title(r"ramp filtered acquired sinogram $y \ast h$", fontsize="medium")
ax6[1, 1].set_title(
    r"back projection of filtered sinogram $R^T(y \ast h)$", fontsize="medium"
)
fig6.colorbar(im06, ax=ax6[0, 0], location="bottom", fraction=0.03)
fig6.colorbar(im16, ax=ax6[0, 1], location="bottom", fraction=0.03)
fig6.colorbar(im26, ax=ax6[1, 0], location="bottom", fraction=0.03)
fig6.colorbar(im36, ax=ax6[1, 1], location="bottom", fraction=0.03)

Let's also reconstruct the noisy sinogram using 100 MLEM iterations.

In [ ]:
# allocate an array for all 50 MLEm updates and the initialization 
mlem_iterates_noisy = np.zeros((num_iterations + 1,) + discrete_radon_transform.image_shape, dtype=np.float32)

# initialize with the 0-th MLEM iterate with a disk full 0.5
X, Y = np.meshgrid(discrete_radon_transform.pixel_x_coordinates, discrete_radon_transform.pixel_y_coordinates, indexing="ij")
R = np.sqrt(X**2 + Y**2)
mlem_iterates_noisy[0, ...][R <= discrete_radon_transform.s.max()] = 0.5

# run the MLEM updates
for i in range(num_iterations):
    print(f"{(i+1):03}/{num_iterations:03}", end="\r")
    mlem_iterates_noisy[i + 1, ...] = my_mlem_update(
        current_image=mlem_iterates_noisy[i, ...],
        data_sinogram=noisy_sinogram1,
        linear_physics_operator=discrete_radon_transform,
        sensitivity_image=sens_image,
    )

In [ ]:
# show the MLEM iterates
fig7, ax7 = plt.subplots(
    4, 4, figsize=(12, 12), tight_layout=True, sharex="row", sharey="row"
)

# show a few selected MLEM updates
for i, it in enumerate([0, 1, 2, 3, 4, 5, 6, 7, 10, 20, 40, num_iterations]):
    im = ax7.ravel()[i].imshow(mlem_iterates_noisy[it].T, **image_plot_kws, vmax=np.percentile(mlem_iterates_noisy[-1,...],99.9))
    ax7.ravel()[i].set_title(f"MLEM iterate {it}", fontsize="medium")
    ax7.ravel()[i].set_xlabel(discrete_radon_transform.image_axis_labels[0])
    ax7.ravel()[i].set_ylabel(discrete_radon_transform.image_axis_labels[1])
    fig7.colorbar(im, ax=ax7.ravel()[i], location="bottom", fraction=0.03, pad = 0.2)

# show the converges of the in two selected pixels
ax7[-1, 0].plot(mlem_iterates_noisy[:, 100, 115])
ax7[-1, 1].plot(mlem_iterates_noisy[:, 67, 100])

for axx in ax7[-1, :2]:
    axx.set_xlabel("MLEM iteration")
    axx.set_ylabel("pixel value")
    axx.grid(ls=":")
ax7[-1, 0].set_title("convergence pixel (100, 115)", fontsize="medium")
ax7[-1, 1].set_title("convergence pixel (67, 100)", fontsize="medium")
for axx in ax7[-1, 2:]:
    axx.set_axis_off()

## Task 6.1

(6.1) Describe the differences between the filtered back projection and MLEM (after 100 iterations) reconstructions of the noisy sinogram.

YOUR ANSWER HERE

## Task 6.2

(6.2) Describe the differences between the early (5-10) and later MLEM iterates (40-100) in terms of spatial resolution and noise and explain why that is.

YOUR ANSWER HERE